In [4]:
import subprocess
import re

def get_chrome_version():
    """Get the installed Chrome version on Windows."""
    try:
        # Try to get Chrome version from registry
        result = subprocess.run([
            'reg', 'query', 
            'HKEY_CURRENT_USER\\Software\\Google\\Chrome\\BLBeacon',
            '/v', 'version'
        ], capture_output=True, text=True, shell=True)
        
        if result.returncode == 0:
            version_match = re.search(r'version\s+REG_SZ\s+(\d+\.\d+\.\d+\.\d+)', result.stdout)
            if version_match:
                return version_match.group(1)
    except:
        pass
    
    # Alternative method - check Chrome executable
    try:
        chrome_paths = [
            r"C:\Program Files\Google\Chrome\Application\chrome.exe",
            r"C:\Program Files (x86)\Google\Chrome\Application\chrome.exe"
        ]
        
        for chrome_path in chrome_paths:
            if os.path.exists(chrome_path):
                result = subprocess.run([chrome_path, '--version'], 
                                      capture_output=True, text=True)
                if result.returncode == 0:
                    version_match = re.search(r'(\d+\.\d+\.\d+\.\d+)', result.stdout)
                    if version_match:
                        return version_match.group(1)
    except:
        pass
    
    return None

print("Checking your Chrome version...")
chrome_version = get_chrome_version()
if chrome_version:
    print(f"✅ Chrome version detected: {chrome_version}")
    major_version = chrome_version.split('.')[0]
    print(f"🎯 You need ChromeDriver for Chrome {major_version}")
else:
    print("❌ Could not detect Chrome version automatically")
    print("Please check manually: Chrome → Help → About Google Chrome")

Checking your Chrome version...
✅ Chrome version detected: 137.0.7151.69
🎯 You need ChromeDriver for Chrome 137


In [7]:
import os
import requests
import zipfile
import json
from pathlib import Path

def download_chromedriver(chrome_version=None):
    """Download and install ChromeDriver for the current Chrome version."""
    
    print("🔍 Finding the correct ChromeDriver version...")
    
    # ChromeDriver version mapping URL
    try:
        # Get the latest ChromeDriver versions
        response = requests.get("https://googlechromelabs.github.io/chrome-for-testing/known-good-versions-with-downloads.json")
        data = response.json()
        
        if chrome_version:
            major_version = chrome_version.split('.')[0]
            print(f"Looking for ChromeDriver compatible with Chrome {major_version}...")
            
            # Find compatible version
            compatible_version = None
            for version_info in reversed(data['versions']):
                if version_info['version'].startswith(major_version + '.'):
                    compatible_version = version_info
                    break
            
            if not compatible_version:
                print(f"❌ No compatible ChromeDriver found for Chrome {major_version}")
                return False
                
            chromedriver_version = compatible_version['version']
        else:
            # Get the latest stable version
            chromedriver_version = data['versions'][-1]['version']
            
        print(f"📦 Will download ChromeDriver version: {chromedriver_version}")
        
        # Find the Windows download URL
        download_url = None
        for version_info in data['versions']:
            if version_info['version'] == chromedriver_version:
                for download in version_info['downloads'].get('chromedriver', []):
                    if download['platform'] == 'win64':
                        download_url = download['url']
                        break
                break
        
        if not download_url:
            print("❌ Could not find Windows ChromeDriver download URL")
            return False
            
        print(f"📥 Downloading from: {download_url}")
        
        # Download ChromeDriver
        response = requests.get(download_url)
        zip_path = Path("chromedriver.zip")
        
        with open(zip_path, 'wb') as f:
            f.write(response.content)
        
        # Extract ChromeDriver
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(".")
        
        # Find the extracted chromedriver.exe
        chromedriver_exe = None
        for root, dirs, files in os.walk("."):
            for file in files:
                if file == "chromedriver.exe":
                    chromedriver_exe = os.path.join(root, file)
                    break
            if chromedriver_exe:
                break
        
        if chromedriver_exe and os.path.exists(chromedriver_exe):
            # Move to project directory
            target_path = Path("chromedriver.exe")
            if target_path.exists():
                target_path.unlink()  # Remove existing
            
            os.rename(chromedriver_exe, target_path)
            
            # Cleanup
            zip_path.unlink()
            # Remove extracted folders
            for item in Path(".").iterdir():
                if item.is_dir() and "chromedriver" in item.name.lower():
                    import shutil
                    shutil.rmtree(item)
            
            print(f"✅ ChromeDriver installed successfully at: {target_path.absolute()}")
            print(f"🎯 Version: {chromedriver_version}")
            return True
        else:
            print("❌ Failed to extract ChromeDriver")
            return False
            
    except Exception as e:
        print(f"❌ Error downloading ChromeDriver: {e}")
        return False

# Run the ChromeDriver installation
print("🚀 Starting ChromeDriver installation...")
success = download_chromedriver("137")

if success:
    print("\n" + "="*50)
    print("✅ CHROMEDRIVER INSTALLATION COMPLETE!")
    print("="*50)
    print("ChromeDriver has been installed in your project directory.")
    print("You can now run the Prezi scraper!")
else:
    print("\n" + "="*50)  
    print("❌ AUTOMATIC INSTALLATION FAILED")
    print("="*50)
    print("Please install ChromeDriver manually using the steps below.")

🚀 Starting ChromeDriver installation...
🔍 Finding the correct ChromeDriver version...
Looking for ChromeDriver compatible with Chrome 137...
📦 Will download ChromeDriver version: 137.0.7151.68
📥 Downloading from: https://storage.googleapis.com/chrome-for-testing-public/137.0.7151.68/win64/chromedriver-win64.zip
✅ ChromeDriver installed successfully at: c:\Users\mkof6\Documents\PythonProjects\utility projects\prezi_download\chromedriver.exe
🎯 Version: 137.0.7151.68

✅ CHROMEDRIVER INSTALLATION COMPLETE!
ChromeDriver has been installed in your project directory.
You can now run the Prezi scraper!
